# California Housing Dataset - Exploratory Data Analysis

Dans ce notebook, nous allons explorer le dataset California Housing pour comprendre :
- La distribution des variables
- Les corrélations entre features
- La présence d'outliers
- La skewness des distributions

Cette analyse nous aidera à définir notre stratégie de preprocessing.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_california_housing

%matplotlib inline
plt.style.use('seaborn')
sns.set_palette('husl')

## 1. Chargement des Données

In [ ]:
# Chargement du dataset
data = fetch_california_housing(as_frame=True)
df = data.frame.copy()

print("Dimensions du dataset:", df.shape)
display(df.head())
print("\nDescription des features:")
for name, desc in zip(data.feature_names, data.feature_names_original):
    print(f"- {name}: {desc}")

## 2. Analyse Statistique

In [ ]:
# Statistiques descriptives
display(df.describe())

# Vérification des valeurs manquantes
missing = df.isnull().sum()
if missing.any():
    print("\nValeurs manquantes:")
    print(missing[missing > 0])
else:
    print("\nAucune valeur manquante !")

## 3. Analyse des Distributions

In [ ]:
# Distribution des variables
fig, axes = plt.subplots(4, 2, figsize=(15, 20))
axes = axes.ravel()

for idx, col in enumerate(df.columns):
    sns.histplot(data=df, x=col, ax=axes[idx])
    axes[idx].set_title(f'Distribution de {col}')

plt.tight_layout()
plt.show()

# Analyse de la skewness
skewness = df.skew().sort_values(ascending=False)
print("\nSkewness des variables:")
print(skewness)

## 4. Analyse des Outliers

In [ ]:
# Boxplots pour détecter les outliers
fig, axes = plt.subplots(4, 2, figsize=(15, 20))
axes = axes.ravel()

for idx, col in enumerate(df.columns):
    sns.boxplot(data=df, y=col, ax=axes[idx])
    axes[idx].set_title(f'Boxplot de {col}')

plt.tight_layout()
plt.show()

# Calcul du pourcentage d'outliers par variable
def detect_outliers_iqr(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    return ((column < (Q1 - 1.5 * IQR)) | (column > (Q3 + 1.5 * IQR))).sum() / len(column) * 100

outliers_pct = pd.Series({col: detect_outliers_iqr(df[col]) for col in df.columns})
print("\nPourcentage d'outliers par variable:")
print(outliers_pct.sort_values(ascending=False))

## 5. Analyse des Corrélations

In [ ]:
# Matrice de corrélation
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Matrice de Corrélation')
plt.show()

# Corrélations avec la variable cible
target_corr = df.corr()['MedHouseVal'].sort_values(ascending=False)
print("\nCorrélations avec MedHouseVal:")
print(target_corr)

## 6. Visualisations Avancées

In [ ]:
# Pairplot des variables les plus corrélées avec la cible
top_features = target_corr[1:4].index  # Top 3 features
sns.pairplot(df, vars=list(top_features) + ['MedHouseVal'])
plt.show()

## 7. Conclusions

De cette analyse exploratoire, nous pouvons conclure :

1. Distribution des données :
   - Certaines variables présentent une forte asymétrie
   - Présence d'outliers significatifs

2. Corrélations :
   - Les features les plus corrélées avec le prix
   - Potentielles multicolinéarités

3. Stratégie de preprocessing suggérée :
   - Traitement des outliers
   - Transformation des variables skewed
   - Standardisation des features